In [3]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import random
import math
import os
import glob
from sklearn.cluster import SpectralClustering 
!pip install pymatreader
from pymatreader import read_mat
from PIL import Image

# Visualize the image and the ground truth segmentation

In [2]:
def draw_ground_truth(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
    seg_path = img_path.replace("images", "ground_truth")
    seg_path = seg_path.replace("jpg", "mat")
    seg = read_mat(seg_path)
    for i in range(len(seg['groundTruth'])):
        mask = np.zeros(img.shape, np.uint8)
       # mask[seg['groundTruth'][i]['Boundaries']==1]=255#Segmentation
        
        for j in range(len(np.unique(seg['groundTruth'][i]['Segmentation']))):
            mask[seg['groundTruth'][i]['Segmentation']==j,0]=j*100
            mask[seg['groundTruth'][i]['Segmentation']==j,1]=j*50
            mask[seg['groundTruth'][i]['Segmentation']==j,2]=j*150
        #print(len(np.unique(seg['groundTruth'][i]['Segmentation'])))

        plt.imshow(mask)
        plt.show()

In [4]:
draw_ground_truth('../input/berkeley-segmentation-dataset-500-bsds500/images/test/100039.jpg')

# Segmentation using K-means

In [4]:
def k_means(image,k,it_max):
    C = []
    for i in range(k):
        C.append(np.array([random.randint(0, 255),random.randint(0, 255),random.randint(0, 255)]))
    C
    for x in range(it_max):
        #print("iteration, ",x)
        clusters = []
        for i in range(k):
            clusters.append([])

        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                minDis = float('inf')
                minC = -1
                for c in range(k):
                    diff = (image[i,j,:].astype('int16')-C[c].astype('int16'))
                    dis = np.dot(diff,diff)
                    if dis<minDis:
                        minDis = dis
                        minC = c
                clusters[minC].append(image[i,j,:])
        for c in range(k):
            newC = np.array([0,0,0])
            for i in range(len(clusters[c])):
                newC+=clusters[c][i]
            if len(clusters[c])==0:
                clusters[c] = 1
            for i in range(3):
                newC[i]/=len(clusters[c])
            same = 1;
            for c1 in range(3):
                if C[c][c1]!= newC[c1]:
                    same=0
            C[c] = newC
        if same:
                return C
    return C

In [5]:
def draw_clusters(image,clusters):
   # print(len(clusters))
    img_clusters = []
    for c in range(len(clusters)):
        img_clusters.append([])

    mask = np.zeros(image.shape, np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            minDis = float('inf')
            minC = -1
            for c in range(len(clusters)):
                diff = (image[i,j,:].astype('int16')-clusters[c].astype('int16'))
                dis = np.dot(diff,diff)
                if dis<minDis:
                    minDis = dis
                    minC = c
            mask[i,j]=clusters[minC]
            img_clusters[minC].append(np.array([i,j]))
    #plt.imshow(mask)
    #plt.show()
    return mask ,img_clusters

In [6]:
def evaluate_entropy(img_path,clusters):
    seg_path = img_path.replace("images", "ground_truth")
    seg_path = seg_path.replace("jpg", "mat")
    data = read_mat(seg_path)
    data = data['groundTruth']
    H_total = 0
    img_size = data[0]['Segmentation'].shape[0]*data[0]['Segmentation'].shape[1]
    
    for x in range(len(data)):
        ground_truth = data[x]['Segmentation']
        HC = 0
        for c in range(len(clusters)):
            nij = np.zeros(len(np.unique(ground_truth)))
            for i in range(len(clusters[c])):
                d = [clusters[c][i]]
                nij[ground_truth[d[0][0],d[0][1]]-1] += 1
            n  = nij/len(clusters[c])
            HCi = 0
            for i in range(len(np.unique(ground_truth))):
                if(n[i]!=0):
                    HCi -= (n[i])* math.log2(n[i])
            #print(HCi," ",n)
            HC += HCi*(len(clusters[c])/img_size)
        #print(HC)
        H_total += HC
    H_total /= len(data)
    return H_total

In [7]:
def evaluate_F_measure(img_path,clusters):
    seg_path = img_path.replace("images", "ground_truth")
    seg_path = seg_path.replace("jpg", "mat")
    data = read_mat(seg_path)
    data = data['groundTruth']
    F_total = 0
    for x in range(len(data)):
        F=0.0
        Tj = np.zeros(len(np.unique(data[x]['Segmentation'])))
        ground_truth = data[x]['Segmentation']
        h = 0
        for i in range(ground_truth.shape[0]):
            for j in range(ground_truth.shape[1]):
                Tj[ground_truth[i,j]-1] += 1
        for c in range(len(clusters)):
            nij = np.zeros(len(np.unique(ground_truth)))
            for i in range(len(clusters[c])):
                d = [clusters[c][i]]
                nij[ground_truth[d[0][0],d[0][1]]-1] += 1
            maxI = -1
            maxN = -1
            for i in range(len(nij)):
                N = nij[i]
                if N>maxN:
                    maxN = N
                    maxI = i
            prec = (maxN/len(clusters[c]))
            recall = (maxN/Tj[maxI])
            F += (2*prec*recall)/(prec+recall) 
    #         print("p",prec)
    #         print("r",recall)
        F /= len(clusters)
        #print(F)
        F_total += F
    F_total /= len(data)
    return F_total

In [ ]:
best_clusters = []
for k in range(1,12,2):
    best_clusters.append([])

dataset_average = np.zeros([7,2])
for i in range(1,51):
    print("img ",i,":", list_of_files[i][63:])
    for k in range(1,12,2):
        print("for k = ",k)
        best = [-1,-1]
        best_cluster = []
        for j in range(3):
            img = cv2.imread(list_of_files[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_clusters = k_means(img,k,10)
            im,clusters=draw_clusters(img,img_clusters)
            f = evaluate_F_measure(list_of_files[i],clusters)
            e = evaluate_entropy(list_of_files[i],clusters)
            if(f > best[0]):
                best[0]=f
                best[1]=e
                best_cluster = img_clusters
        print("best F-measure = ",best[0])
        print("best Conditional Entropy = ",best[1])
        dataset_average[k//2,0]+=best[0]
        dataset_average[k//2,1]+=best[1]
        best_clusters[k//2].append(best_cluster)
        
dataset_average/=10
print("data_set average")
for k in range(1,12,2):
    print("for k = ",k," , average F-measure = ",dataset_average[k//2,0])
    print("        ",k," , average Conditional Entropy = ",dataset_average[k//2,1])

# big picture

# img 1 

In [11]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/108004.jpg'
draw_ground_truth(img_path)
img = Image.open(img_path)
img = np.array(img)

In [13]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means(img,5,10)
    im,clusters=draw_clusters(img,img_clusters)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

In [18]:
#Normalized-cut for the 5-NN graph,
img1d = img.reshape(img.shape[0]*img.shape[1],3)
clusters = SpectralClustering(n_clusters=5,affinity='nearest_neighbors',n_neighbors=5,n_jobs=-1).fit_predict(img1d)
img_cluster = np.zeros(img1d.shape,dtype=np.uint8)
for i in range(img1d.shape[0]):
    img_cluster[i] = [clusters[i]*100,clusters[i]*50,clusters[i]*150]
plt.imshow(img_cluster.reshape(img.shape[0],img.shape[1],3))
plt.figure()

# img 2

In [22]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/15011.jpg'
draw_ground_truth(img_path)
img = Image.open(img_path)
img = np.array(img)

In [ ]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means(img,5,10)
    im,clusters=draw_clusters(img,img_clusters)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

In [ ]:
#Normalized-cut for the 5-NN graph,
img1d = img.reshape(img.shape[0]*img.shape[1],3)
clusters = SpectralClustering(n_clusters=5,affinity='nearest_neighbors',n_neighbors=5,n_jobs=-1).fit_predict(img1d)
img_cluster = np.zeros(img1d.shape,dtype=np.uint8)
for i in range(img1d.shape[0]):
    img_cluster[i] = [clusters[i]*100,clusters[i]*50,clusters[i]*150]
plt.imshow(img_cluster.reshape(img.shape[0],img.shape[1],3))
plt.figure()

# img 3

In [50]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/41085.jpg'
draw_ground_truth(img_path)
img = Image.open(img_path)
img = np.array(img)

In [ ]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means(img,5,10)
    im,clusters=draw_clusters(img,img_clusters)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

In [ ]:
#Normalized-cut for the 5-NN graph,
img1d = img.reshape(img.shape[0]*img.shape[1],3)
clusters = SpectralClustering(n_clusters=5,affinity='nearest_neighbors',n_neighbors=5,n_jobs=-1).fit_predict(img1d)
img_cluster = np.zeros(img1d.shape,dtype=np.uint8)
for i in range(img1d.shape[0]):
    img_cluster[i] = [clusters[i]*100,clusters[i]*50,clusters[i]*150]
plt.imshow(img_cluster.reshape(img.shape[0],img.shape[1],3))
plt.figure()

# img 4

In [32]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/306052.jpg'
draw_ground_truth(img_path)
img = Image.open(img_path)
img = np.array(img)

In [ ]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means(img,5,10)
    im,clusters=draw_clusters(img,img_clusters)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

In [ ]:
#Normalized-cut for the 5-NN graph,
img1d = img.reshape(img.shape[0]*img.shape[1],3)
clusters = SpectralClustering(n_clusters=5,affinity='nearest_neighbors',n_neighbors=5,n_jobs=-1).fit_predict(img1d)
img_cluster = np.zeros(img1d.shape,dtype=np.uint8)
for i in range(img1d.shape[0]):
    img_cluster[i] = [clusters[i]*100,clusters[i]*50,clusters[i]*150]
plt.imshow(img_cluster.reshape(img.shape[0],img.shape[1],3))
plt.figure()

# img 5

In [ ]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/344010.jpg'
draw_ground_truth(img_path)
img = Image.open(img_path)
img = np.array(img)

In [ ]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means(img,5,10)
    im,clusters=draw_clusters(img,img_clusters)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

In [ ]:
#Normalized-cut for the 5-NN graph,
img1d = img.reshape(img.shape[0]*img.shape[1],3)
clusters = SpectralClustering(n_clusters=5,affinity='nearest_neighbors',n_neighbors=5,n_jobs=-1).fit_predict(img1d)
img_cluster = np.zeros(img1d.shape,dtype=np.uint8)
for i in range(img1d.shape[0]):
    img_cluster[i] = [clusters[i]*100,clusters[i]*50,clusters[i]*150]
plt.imshow(img_cluster.reshape(img.shape[0],img.shape[1],3))
plt.figure()

# bonus

In [48]:
def k_means_spatial(image,k,it_max):
    C = []
    for i in range(k):
        C.append(np.array([random.randint(0, 255),random.randint(0, 255),random.randint(0, 255),
                          random.randint(0, image.shape[0]),random.randint(0, image.shape[1])]))
    for x in range(it_max):
        clusters = []
        for i in range(k):
            clusters.append([])

        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                minDis = float('inf')
                minC = -1
                for c in range(k):
                    diff = (image[i,j,:].astype('int16')-C[c].astype('int16'))
                    dis = np.dot(diff,diff)
                    if dis<minDis:
                        minDis = dis
                        minC = c
                clusters[minC].append(image[i,j,:])
        for c in range(k):
            newC = np.array([0,0,0,0,0])
            for i in range(len(clusters[c])):
                newC+=clusters[c][i]
            for i in range(5):
                newC[i]/=len(clusters[c])
            same = 1;
            for c1 in range(5):
                if C[c][c1]!= newC[c1]:
                    same=0
            C[c] = newC
        if same:
            return C
    return C

# img 1

In [52]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/108004.jpg'
img = Image.open(img_path)
img = np.array(img)
plt.imshow(img)
plt.show()

In [53]:
img_with_spatial = np.empty((img.shape[0],img.shape[1],5))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_with_spatial[i,j,:]=[img[i,j,0],img[i,j,1],img[i,j,2],i,j]
img_with_spatial= img_with_spatial.astype('int16')

In [54]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means_spatial(img_with_spatial,5,10)
    c = []
    for i in range(5):
        c.append(img_clusters[i][0:3])
    im,clusters=draw_clusters(img,c)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

# img2

In [61]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/15011.jpg'
img = Image.open(img_path)
img = np.array(img)
plt.imshow(img)
plt.show()

In [62]:
img_with_spatial = np.empty((img.shape[0],img.shape[1],5))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_with_spatial[i,j,:]=[img[i,j,0],img[i,j,1],img[i,j,2],i,j]
img_with_spatial= img_with_spatial.astype('int16')

In [63]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means_spatial(img_with_spatial,5,10)
    c = []
    for i in range(5):
        c.append(img_clusters[i][0:3])
    im,clusters=draw_clusters(img,c)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

# img3

In [64]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/41085.jpg'
img = Image.open(img_path)
img = np.array(img)
plt.imshow(img)
plt.show()

In [65]:
img_with_spatial = np.empty((img.shape[0],img.shape[1],5))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_with_spatial[i,j,:]=[img[i,j,0],img[i,j,1],img[i,j,2],i,j]
img_with_spatial= img_with_spatial.astype('int16')

In [66]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means_spatial(img_with_spatial,5,10)
    c = []
    for i in range(5):
        c.append(img_clusters[i][0:3])
    im,clusters=draw_clusters(img,c)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

# img4

In [67]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/306052.jpg'
img = Image.open(img_path)
img = np.array(img)
plt.imshow(img)
plt.show()

In [68]:
img_with_spatial = np.empty((img.shape[0],img.shape[1],5))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_with_spatial[i,j,:]=[img[i,j,0],img[i,j,1],img[i,j,2],i,j]
img_with_spatial= img_with_spatial.astype('int16')

In [69]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means_spatial(img_with_spatial,5,10)
    c = []
    for i in range(5):
        c.append(img_clusters[i][0:3])
    im,clusters=draw_clusters(img,c)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()

# img5

In [70]:
img_path = '../input/berkeley-segmentation-dataset-500-bsds500/images/test/344010.jpg'
img = Image.open(img_path)
img = np.array(img)
plt.imshow(img)
plt.show()

In [71]:
img_with_spatial = np.empty((img.shape[0],img.shape[1],5))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_with_spatial[i,j,:]=[img[i,j,0],img[i,j,1],img[i,j,2],i,j]
img_with_spatial= img_with_spatial.astype('int16')

In [72]:
best = [-1,-1]
best_cluster = []
for j in range(3):
    img_clusters = k_means_spatial(img_with_spatial,5,10)
    c = []
    for i in range(5):
        c.append(img_clusters[i][0:3])
    im,clusters=draw_clusters(img,c)
    f = evaluate_F_measure(img_path,clusters)
    e = evaluate_entropy(img_path,clusters)
    if(f > best[0]):
        best[0]=f
        best[1]=e
        best_cluster = img_clusters
print("best F-measure = ",best[0])
print("best Conditional Entropy = ",best[1])
plt.imshow(im)
plt.show()